Data Load

In [1]:
!wget -O LLM-ReDial-2024.zip "https://www.dropbox.com/scl/fi/x9avfdx2a1k6uq97f0efj/LLM-ReDial-2024.zip?rlkey=ijqpf91d13d6lowek3ebjvd0n&e=2&dl=1"
!unzip LLM-ReDial-2024.zip
!unzip LLM_Redial.zip
!rm LLM_Redial.zip
!rm LLM-ReDial-2024.zip
!rm -rf __MACOSX

--2024-12-08 23:32:51--  https://www.dropbox.com/scl/fi/x9avfdx2a1k6uq97f0efj/LLM-ReDial-2024.zip?rlkey=ijqpf91d13d6lowek3ebjvd0n&e=2&dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.81.18, 2620:100:6031:18::a27d:5112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.81.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://uca6be5038f8dc78b0d23a2fbb95.dl.dropboxusercontent.com/cd/0/inline/Cf4MY3qHhn42tnUEw8JHXzHECTq76Vb4zvx6yKgLTV9lcAj8ncsJ1tU2t_5MMZCKHnzcmjdRLSZpifek01nEyzo3427aN3O8rNoqNQPXYZnMwWnJzDMheF73LRmKM5o19Fs/file?dl=1# [following]
--2024-12-08 23:32:52--  https://uca6be5038f8dc78b0d23a2fbb95.dl.dropboxusercontent.com/cd/0/inline/Cf4MY3qHhn42tnUEw8JHXzHECTq76Vb4zvx6yKgLTV9lcAj8ncsJ1tU2t_5MMZCKHnzcmjdRLSZpifek01nEyzo3427aN3O8rNoqNQPXYZnMwWnJzDMheF73LRmKM5o19Fs/file?dl=1
Resolving uca6be5038f8dc78b0d23a2fbb95.dl.dropboxusercontent.com (uca6be5038f8dc78b0d23a2fbb95.dl.dropboxusercontent.com)... 162.125.81.15, 2620:100:6031:15::

In [2]:
from huggingface_hub import notebook_login

notebook_login()

In [3]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-6bd63bd3-af3a-8274-8866-2fca50d10873)


In [4]:
!pip install transformers[torch]
!pip install bitsandbytes
!pip install peft
!pip install sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 11.7 MB/s eta 0:00:00


In [5]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline

In [6]:
model_id = "meta-llama/Llama-3.2-3B-Instruct"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})

tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/878 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

In [7]:
text_generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

In [23]:
def generate_zeroshot(conversation_id, conversations):
    conversation = "\n".join(conversations[conversation_id]["messages"])
    content = (
        "Pretend you are a movie recommender system. "
        "I will give you a conversation between a human and assistant. "
        "Based on the conversation, you will think step-by-step on what information is relevant to the recommendation and then reply with exactly 10 recommendations "
        "including only their titles without extra sentences or description. "
        f"Here is the conversation: {conversation}."
    )
    chat = [
        {"role": "user", "content": content}
    ]
    return tokenizer.apply_chat_template(chat, tokenize=False)

In [24]:
from Tools import read_user_data
path_movie_data = "/content/Movie/final_data.jsonl"

In [25]:
import json
item_map_path = "/content/Movie/item_map.json"
item_map = json.load(open(item_map_path,'r'))
user_map_path = "/content/Movie/user_ids.json"
user_map = json.load(open(user_map_path,'r'))

In [26]:
conversations_path = "/content/Movie/Conversation.txt"

conversations = []
current_conversation = []
conversation_id = 0
with open(conversations_path, 'r') as file:
    for line in file:
        line = line.strip()
        if not line:
            continue

        if line.isdigit():
            if current_conversation:
                conversations.append({
                    'conversation_id': conversation_id,
                    'messages': current_conversation
                })
            conversation_id = int(line)
            current_conversation = []
        else:
            current_conversation.append(line)

    if current_conversation:
        conversations.append({
            'conversation_id': conversation_id,
            'messages': current_conversation
        })

In [27]:
import random
user_ids = list(user_map.keys())
sample_percent = 0.1 # 10%

sample_size = int(sample_percent * len(user_ids))
sampled_users = random.sample(user_ids,sample_size)

In [28]:
dataset = []
for user_id in sampled_users:
  user_data = read_user_data(path_movie_data, user_id)
  for conv_number, conversation in enumerate(user_data['Conversation']):
    conversation_id = conversation[f"conversation_{conv_number+1}"]["conversation_id"]
    prompt = generate_zeroshot(conversation_id, conversations)
    dataset.append({
        "user_id": user_id,
        "conversation_id": conversation_id,
        "user_might_like": user_data['user_might_like'],
        "prompt": prompt
    })


In [29]:
from sklearn.model_selection import train_test_split

train_set, test_set = train_test_split(dataset, test_size=0.2, random_state=42)
print("Train prompts:", len(train_set))
print("Test prompts:", len(test_set))

Train prompts: 933
Test prompts: 234


In [30]:
first10prompts = [item['prompt'] for item in train_set[:10]]
first10results = text_generator(first10prompts, max_new_tokens=1000, return_full_text=False)

In [31]:
first10answers = [result[0]['generated_text'] for result in first10results]
for answer in first10answers:
  print("----------------------")
  print(answer)
  print("----------------------")

----------------------
assistant

Based on the conversation, I've identified the following relevant information:

- Disappointing movie: "Shadow of the Vampire"
- Avoided movies: "Ghost Rider: Spirit of Vengeance", "Snow Shark"
- Recommended movie: "Mongol: The Early Years of Genghis Khan"

Here are 10 movie recommendations:

1. 300
2. The New World
3. The Count of Monte Cristo
4. The Last Samurai
5. The Patriot
6. The Eagle
7. The Conqueror
8. Genghis Khan
9. The Last King of Scotland
10. Alexander
----------------------
----------------------
assistant

Based on the conversation, I've identified the following relevant information:

- The user enjoys biographical dramas and highly praises "Ray".
- The user has also seen "Collateral" and found it thrilling.
- The user is interested in a new recommendation.
- The assistant suggests "United 93" based on the user's interest in "Ray" and "Collateral".

Here are 10 movie recommendations for you:

1. The Pursuit of Happyness
2. Coach Carter


In [42]:
for answer in first10answers:
  print(parse_recommendations(answer))

['300', 'The New World', 'The Count of Monte Cristo', 'The Last Samurai', 'The Patriot', 'The Eagle', 'The Conqueror', 'Genghis Khan', 'The Last King of Scotland', 'Alexander']
['The Pursuit of Happyness', 'Coach Carter', 'The Imitation Game', 'Erin Brockovich', 'The Butler', 'Coach Carter', 'The Social Network', 'Invictus', 'The I Am Malala', 'Coach Carter']
['The Boy Who Could Fly', 'The Little Mermaid II: Return to the Sea', "Charlie's Angels", "Wanda Sykes: I'ma Be Me", 'The Little Mermaid', 'The Boy Who Lived', 'The Princess Diaries', 'Freaky Friday', 'The Parent Trap', 'Freaky Friday']
['The Girl with All the Gifts', 'Ex Machina', 'The Island of the Lost Souls', 'The Bourne Identity', 'The Girl with the Dragon Tattoo', 'The Maze Runner', 'Gattaca', 'eXistenZ', 'The Island', 'Minority Report']
['Dragonslayer', 'The Dark Crystal', 'The Last Unicorn', 'The NeverEnding Story', 'The Last Unicorn', 'The Dark Crystal', 'The NeverEnding Story', 'The Dark Crystal', 'The Sword in the Stone

In [43]:
prompts = [item['prompt'] for item in test_set]
results = text_generator(prompts, max_new_tokens=1000, return_full_text=False)


In [44]:
answers = [result[0]['generated_text'] for result in results]
for answer in answers[:10]:
  print("----------------------")
  print(answer)
  print("----------------------")

----------------------
assistant

Based on the conversation, I will analyze the relevant information and provide 10 movie recommendations. Here are the recommendations:

1. The Lord of the Rings: The Fellowship of the Ring
2. The Shawshank Redemption
3. The Matrix
4. Inception
5. The Dark Knight
6. The Avengers
7. Interstellar
8. The Princess Bride
9. The Silence of the Lambs
10. Star Wars: Episode IV - A New Hope
----------------------
----------------------
assistant

Based on the conversation, I will recommend the following 10 movies:

1. The Shining
2. The Bank Job
3. 1408 Full Screen Edition
4. Alien Trespass
5. How I Met Your Mother
6. Scrubs
7. Friends
8. The Truman Show
9. Eternal Sunshine of the Spotless Mind
10. Dodgeball: A True Underdog Story
----------------------
----------------------
assistant

Based on the conversation, I'll recommend the following 10 movies:

1. A Holiday for Love
2. The Holiday
3. Love Actually
4. The Holiday
5. Love, Actually
6. The Holiday (2006)
7

In [45]:
import re

def parse_recommendations(answer):
  answer = answer.split("\n")
  recommendations = answer[2:len(answer)]
  recommendations = [re.sub(r'\d+\.\s|"', '', recommendation) for recommendation in recommendations]
  return recommendations[-10:]

parse_recommendations(answers[3])

['The Last of the Mohicans',
 'The Big Lebowski',
 'Fargo',
 'No Country for Old Men',
 "The Man Who Wasn't There",
 'The Hudsucker Proxy',
 'The Royal Tenenbaums',
 'The Ice Storm',
 'The Secret Garden',
 'The Grand Budapest Hotel']

In [46]:
less_answers = 0
more_answers = 0
for answer in answers:
  recommendations = parse_recommendations(answer)
  if len(recommendations) < 10:
    less_answers += 1
  elif len(recommendations) > 10:
    more_answers += 1

print(f"Less answers: {less_answers}")
print(f"More answers: {more_answers}")


Less answers: 0
More answers: 0


In [21]:
import numpy as np

def recall_at_k(r, k, m):
    return (np.asarray(r)[:k] != 0).sum() / m

def dcg_at_k(r, k):
    r = np.asfarray(r)[:k]
    if r.size:
        return np.sum(np.subtract(np.power(2, r), 1) / np.log2(np.arange(2, r.size + 2)))
    return 0.

def idcg_at_k(k):
    return dcg_at_k(np.ones(k), k)

def ndcg_at_k(r, k, max_relevant):
    idcg = idcg_at_k(min(k, max_relevant))
    if not idcg:
        return 0.
    return dcg_at_k(r, k) / idcg

In [22]:
from sentence_transformers import SentenceTransformer

def embedding_init():
  sbert = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
  return sbert

def similarity_score(sbert, movies):
  embed1 = sbert.encode(movies[0])
  embed2 = sbert.encode(movies[1])
  return sbert.similarity(embed1,embed2).numpy()[0][0]

In [50]:
def calculate_similarities(sbert, user_might_like, recommendations):
  might_like_embeddings = sbert.encode(user_might_like)
  recommendations_embeddings = sbert.encode(recommendations)
  return sbert.similarity(might_like_embeddings, recommendations_embeddings).numpy()

def calculate_rel_vector(similarities, threshold=0.75):
  return np.any(np.array(similarities) >= threshold, axis=0)

In [51]:
def evaluate_model(dataset, answers, sbert):
    assert len(prompts) == len(answers)

    mean_ndcg = 0
    mean_recall = 0
    recommendations_count = 0
    for i in range(len(answers)):
       recommendations = parse_recommendations(answers[i])
       if len(recommendations) != 10:
         continue
       user_might_like = [item_map[id] for id in dataset[i]["user_might_like"]]
       similarities = calculate_similarities(sbert, user_might_like, recommendations)
       rel_vector = calculate_rel_vector(similarities)
       print(rel_vector)


       mean_ndcg += ndcg_at_k(rel_vector, 10, len(user_might_like))
       mean_recall += recall_at_k(rel_vector, 10, len(user_might_like))
       recommendations_count += 1

    mean_ndcg /= recommendations_count
    mean_recall /= recommendations_count

    return mean_ndcg, mean_recall

In [52]:
sbert = embedding_init()
ndcg, recall = evaluate_model(test_set, answers, sbert)
print(f"NDCG: {ndcg:.4f}")
print(f"Recall: {recall:.4f}")

[ True False False False False False False False False False]
[False False False  True False False False False False False]
[ True  True False  True False False False False False  True]
[False False False False False False False False False False]
[False False False False False False False False False False]
[False False  True False False False False False False False]
[False False False False False False False False False False]
[False False False False False False  True False False False]
[False False False False False False False False False False]
[ True False False False False False False False  True False]
[False False False False False False False False False False]
[False False False False False False False False False False]
[False False False False False False False False False False]
[False False False False False  True False False False False]
[False False False False False False False False False False]
[False False False False False False False False False False]
[False F